In [2]:
import pandas as pd

In [3]:
base_dir = 'C:/Users/User/projects/msc-thesis-recurrent-health-modeling/data/mimic-api-demo/'

admissions_df = pd.read_csv(base_dir + 'admissions.csv')
diagnoses_df = pd.read_csv(base_dir + 'diagnoses.csv')
icu_stays_df = pd.read_csv(base_dir + 'icu_stays.csv')
procedures_df = pd.read_csv(base_dir + 'procedures.csv')
prescriptions_df = pd.read_csv(base_dir + 'prescriptions.csv')
patients_metadata_df = pd.read_csv(base_dir + 'patients.csv')
targets_df = pd.read_csv(base_dir + 'targets.csv')

In [4]:
admissions_df.head()

,HADM_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DISCHARGE_LOCATION
0,182104,36,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,Medicare,WHITE,HOME HEALTH CARE
1,122659,36,2131-05-12 19:49:00,2131-05-25 13:30:00,EMERGENCY,Medicare,WHITE,REHAB/DISTINCT PART HOSP
2,191941,107,2115-02-20 17:41:00,2115-02-21 16:30:00,EMERGENCY,Private,HISPANIC OR LATINO,HOME
3,182383,107,2121-11-30 19:24:00,2121-12-05 14:18:00,EMERGENCY,Medicare,HISPANIC OR LATINO,HOME HEALTH CARE
4,145243,222,2137-07-15 15:31:00,2137-07-17 12:00:00,EMERGENCY,Medicare,WHITE,HOME


In [5]:
mask = (targets_df['READMISSION_30_DAYS'] == 1)
targets_df.loc[mask].head()

,SUBJECT_ID,HADM_ID,READMISSION_30_DAYS,DAYS_UNTIL_NEXT_HOSPITALIZATION,READMISSION_TIME_CAT
11,773,160425,1,20.544444,0-30
14,878,102365,1,24.338889,0-30
18,998,171544,1,18.935417,0-30
19,1006,199286,1,12.261111,0-30
20,1050,182917,1,3.957639,0-30


In [6]:
def filter_single_patient(admissions_df, diagnoses_df, icu_stays_df, procedures_df, prescriptions_df, patients_metadata_df, targets_df, subject_id):
    admissions_filtered = admissions_df[admissions_df['SUBJECT_ID'] == subject_id]
    diagnoses_filtered = diagnoses_df[diagnoses_df['SUBJECT_ID'] == subject_id]
    icu_stays_filtered = icu_stays_df[icu_stays_df['SUBJECT_ID'] == subject_id]
    procedures_filtered = procedures_df[procedures_df['SUBJECT_ID'] == subject_id]
    prescriptions_filtered = prescriptions_df[prescriptions_df['SUBJECT_ID'] == subject_id]
    patients_metadata_filtered = patients_metadata_df[patients_metadata_df['SUBJECT_ID'] == subject_id]
    targets_filtered = targets_df[targets_df['SUBJECT_ID'] == subject_id]

    return (admissions_filtered, diagnoses_filtered, icu_stays_filtered, procedures_filtered, prescriptions_filtered, patients_metadata_filtered, targets_filtered)

In [7]:
subject_id_example = 222
filtered_data = filter_single_patient(
    admissions_df,
    diagnoses_df,
    icu_stays_df,
    procedures_df,
    prescriptions_df,
    patients_metadata_df,
    targets_df,
    subject_id_example
)

In [7]:
admissions_single_patient_df = filtered_data[0]
diagnoses_single_patient_df = filtered_data[1]
icu_stays_single_patient_df = filtered_data[2]
procedures_single_patient_df = filtered_data[3]
prescriptions_single_patient_df = filtered_data[4]
patients_metadata_single_patient_df = filtered_data[5]
targets_df_single_patient_df = filtered_data[6]

In [8]:
targets_df_single_patient_df

,SUBJECT_ID,HADM_ID,READMISSION_30_DAYS,DAYS_UNTIL_NEXT_HOSPITALIZATION,READMISSION_TIME_CAT
2,222,188038,0,229.743056,120+


In [9]:
request_payload_single_patient = {
    "admissions": admissions_single_patient_df.to_dict(orient='records'),
    "diagnoses": diagnoses_single_patient_df.to_dict(orient='records'),
    "icu_stays": icu_stays_single_patient_df.to_dict(orient='records'),
    "procedures": procedures_single_patient_df.to_dict(orient='records'),
    "prescriptions": prescriptions_single_patient_df.to_dict(orient='records'),
    "patients": patients_metadata_single_patient_df.to_dict(orient='records'),
    "targets": targets_df_single_patient_df.to_dict(orient='records')
}

In [10]:
request_payload_single_patient

{'admissions': [{'HADM_ID': 145243,
   'SUBJECT_ID': 222,
   'ADMITTIME': '2137-07-15 15:31:00',
   'DISCHTIME': '2137-07-17 12:00:00',
   'ADMISSION_TYPE': 'EMERGENCY',
   'INSURANCE': 'Medicare',
   'ETHNICITY': 'WHITE',
   'DISCHARGE_LOCATION': 'HOME'},
  {'HADM_ID': 105083,
   'SUBJECT_ID': 222,
   'ADMITTIME': '2141-02-18 23:10:00',
   'DISCHTIME': '2141-02-21 15:50:00',
   'ADMISSION_TYPE': 'EMERGENCY',
   'INSURANCE': 'Medicare',
   'ETHNICITY': 'WHITE',
   'DISCHARGE_LOCATION': 'HOME HEALTH CARE'},
  {'HADM_ID': 137006,
   'SUBJECT_ID': 222,
   'ADMITTIME': '2142-06-11 07:15:00',
   'DISCHTIME': '2142-06-19 12:30:00',
   'ADMISSION_TYPE': 'ELECTIVE',
   'INSURANCE': 'Medicare',
   'ETHNICITY': 'WHITE',
   'DISCHARGE_LOCATION': 'HOME HEALTH CARE'},
  {'HADM_ID': 188038,
   'SUBJECT_ID': 222,
   'ADMITTIME': '2142-06-23 05:02:00',
   'DISCHTIME': '2142-07-01 17:30:00',
   'ADMISSION_TYPE': 'EMERGENCY',
   'INSURANCE': 'Medicare',
   'ETHNICITY': 'WHITE',
   'DISCHARGE_LOCATION': 

In [18]:
request_payload_mult_patients = {
    "admissions": admissions_df.to_dict(orient="records"),
    "diagnoses": diagnoses_df.to_dict(orient="records"),
    "icu_stays": icu_stays_df.to_dict(orient="records"),
    "procedures": procedures_df.to_dict(orient="records"),
    "prescriptions": prescriptions_df.to_dict(orient="records"),
    "patients": patients_metadata_df.to_dict(orient="records"),
    "targets": targets_df.to_dict(orient="records")
}

In [12]:
from recurrent_health_events_prediction.api.services.prediction import ModelPrediction

model_prediction_service = ModelPrediction()

In [19]:
reponse_mult_patients = model_prediction_service.predict(request_payload_mult_patients)

In [20]:
reponse_mult_patients['prediction'].keys()

dict_keys(['pred_probs', 'pred_labels', 'true_labels', 'attention_weights', 'hadm_ids', 'subject_ids'])

In [21]:
subject_ids = reponse_mult_patients['prediction']['subject_ids']
hadm_ids = reponse_mult_patients['prediction']['hadm_ids']
pred_probs = reponse_mult_patients['prediction']['pred_probs']
pred_labels = reponse_mult_patients['prediction']['pred_labels']
true_labels = reponse_mult_patients['prediction']['true_labels']
results_df = pd.DataFrame({
    'SUBJECT_ID': subject_ids,
    'HADM_ID': hadm_ids,
    'PRED_PROB': pred_probs,
    'PRED_LABEL': pred_labels,
    'TRUE_LABEL': true_labels
})

In [22]:
results_df

,SUBJECT_ID,HADM_ID,PRED_PROB,PRED_LABEL,TRUE_LABEL
0,36,122659,0.461308,0,0.0
1,107,182383,0.259521,0,0.0
2,222,188038,0.309999,0,0.0
3,236,182562,0.163619,0,0.0
4,291,125726,0.169115,0,0.0
...,...,...,...,...,...
995,99346,177015,0.389370,0,0.0
996,99469,126023,0.242672,0,0.0
997,99503,146847,0.298876,0,0.0
998,99650,158023,0.157831,0,0.0


In [23]:
reponse_mult_patients['metrics']

{'auroc': np.float64(0.6606068185015554),
 'confusion_matrix': [[711, 30], [230, 29]],
 'recall': 0.11196911196911197,
 'accuracy': 0.74,
 'precision': 0.4915254237288136}

In [ ]:
model_prediction_service.predict(request_payload_single_patient)

In [ ]:
import requests
import json

url = "http://127.0.0.1:8000/predict"
headers = {"Content-Type": "application/json"}

response = requests.post(url, headers=headers, data=json.dumps(request_payload_mult_patients))

print(response.status_code)
print(response.json())